# LangChain 实战：服装销售聊天机器人

## 使用 GPT-3.5-turbo 构造销售话术数据

ChatGPT 分享链接：https://chat.openai.com/share/a990d5d9-bad6-4329-b7b7-a90219d39e6c


使用 ChatGPT 构造销售数据的 Prompt 示例：

```
你是中国顶级的服装销售，现在培训职场新人，请给出100条实用的销售话术。

每条销售话术以如下格式给出：
[客户问题]
[销售回答]

```

GPT-3.5-turbo 回复结果：

```
1. [客户问题] “这件衬衫看起来不错，但我不确定适不适合我。”
   [销售回答] “我完全理解您的担忧。我们提供免费试穿服务，您可以试试这款衬衫，确保它与您的体型和风格完美匹配。”

2. [客户问题] “这条裤子的价格有点高，你们能不能给个折扣？”
   [销售回答] “我明白您关注价格的考量。但是考虑到我们的产品质量和设计，这个价格实际上是非常合理的。不过，如果您对其他款式感兴趣，我可以帮您找到一些更优惠的选择。”

3. [客户问题] “我担心这件外套的质量不够好。”
   [销售回答] “我们的外套采用了优质的材料，经过严格的质量控制。此外，我们还提供了一年的质量保证，如果您在使用过程中有任何问题，我们都会为您提供全额退款或换货。”

4. [客户问题] “我对这款服装不是很了解，你能帮我介绍一下吗？”
   [销售回答] “当然！这款服装是我们最畅销的产品之一。它采用了流行的设计和舒适的面料，非常适合各种场合穿着。让我向您详细介绍一下它的特点和穿着方式。”

5. [客户问题] “我想要一套适合商务场合穿着的西装。”
   [销售回答] “绝对没问题！我们有各种款式和颜色的西装供您选择。您想要传统的款式还是更时尚的设计？您的身形和偏好是什么？让我们一起找到最适合您的那一款。”

6. [客户问题] “我对您们的退换货政策有些疑问。”
   [销售回答] “我们有非常灵活的退换货政策，您可以放心购买。如果您对商品不满意或尺码不合适，我们接受无条件退货并提供全额退款。如果您更换其他商品，我们也会尽力满足您的需求。”

7. [客户问题] “我对这款衣服的尺码是否准确有所疑问。”
   [销售回答] “我们的尺码通常很准确，但每个品牌的尺码标准可能略有不同。为了确保您选择的尺码合适，我建议您测量一下自己的身体尺寸，或者您也可以在试穿时请我们的专业顾问为您提供帮助。”

8. [客户问题] “我在其他地方看到了类似的产品价格更低，你们能否匹价？”
   [销售回答] “我们真诚地感谢您对我们产品的关注，并理解您对价格的考量。尽管我们竭尽所能确保提供最具竞争力的价格，但我们也重视产品质量和服务。我们会尽最大努力为您提供更好的购物体验，包括提供专业的服务和优质的售后支持。”

9. [客户问题] “我不太确定这件衣服的颜色是否适合我。”
   [销售回答] “颜色是非常个人化的选择，我完全理解您的顾虑。如果您愿意，我可以为您提供一些搭配建议，或者您也可以试穿这件衣服，看看它在不同光线下的效果如何。”

10. [客户问题] “我希望购买一套适合夏季运动的运动服。”
    [销售回答] “夏季运动服需要轻薄透气，同时具备良好的排汗性能。我们有多种款式和材质的运动服可供选择，您可以根据您的喜好和运动类型来挑选。此外，我们的运动服还有防紫外线和快干功能，确保您在运动中保持舒适。”
```


## 使用 Document Transformers 模块来处理原始数据


将 ChatGPT 生成的结果保存到 [real_estate_sales_data.txt](real_estate_sales_data.txt) 文件中

In [1]:
with open("real_estate_sales_data.txt",encoding = 'utf-8') as f:
    real_estate_sales = f.read()

### 使用 CharacterTextSplitter 来进行文本分割

- 基于单字符来进行文本分割（separator）
- 基于字符数来决定文本块长度（chunk_size）

参考示例：

```python
from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter(        
    separator = "\n\n",
    chunk_size = 1000,
    chunk_overlap  = 200,
    length_function = len,
    is_separator_regex = False,
)
```


In [2]:
from langchain.text_splitter import CharacterTextSplitter

In [3]:
text_splitter = CharacterTextSplitter(        
    separator = r'\d+\.',
    chunk_size = 200,
    chunk_overlap  = 0,
    length_function = len,
    is_separator_regex = True,
)

In [4]:
docs = text_splitter.create_documents([real_estate_sales])

In [5]:
docs[0]

Document(page_content='[客户问题] “这件衬衫看起来不错，但我不确定适不适合我。”\n   [销售回答] “我完全理解您的担忧。我们提供免费试穿服务，您可以试试这款衬衫，确保它与您的体型和风格完美匹配。”')

In [6]:
len(docs)

100

### 使用 Faiss 作为向量数据库，持久化存储房产销售 问答对（QA-Pair）

In [7]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS

db = FAISS.from_documents(docs, OpenAIEmbeddings())

C:\Users\ckc\.conda\envs\langchain\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.0.9 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [8]:
query = "外套质量怎么样"

In [9]:
answer_list = db.similarity_search(query)

In [10]:
for ans in answer_list:
    print(ans.page_content + "\n")

[客户问题] “我担心这件外套的质量不够好。”
   [销售回答] “我们的外套采用了优质的材料，经过严格的质量控制。此外，我们还提供了一年的质量保证，如果您在使用过程中有任何问题，我们都会为您提供全额退款或换货。”

[客户问题] “我对这个款式的外套的保暖性有些疑问，你能介绍一下吗？”
    [销售回答] “这款外套采用了保暖性能较好的面料，内部可能还有保暖填充物，确保在寒冷的天气中保持温暖。您也可以选择适合的内衬来增加保暖效果，让您在冬季穿着更加舒适。”

[客户问题] “我对这个款式的衣服的质地有些担心，你能介绍一下吗？”
    [销售回答] “这款衣服采用了优质的面料，手感柔软舒适，具有良好的透气性和吸湿排汗性能，穿着舒适且质地优良。我们也提供了清洗说明，让您更好地保养衣服。”

[客户问题] “我对这种款式的外套的适用季节有些不确定，你能给我介绍一下吗？”
    [销售回答] “这款外套通常适合春秋季穿着，具有保暖和防风的功能，适合温度稍微凉爽的季节。如果您对适用季节有特殊要求，我们也可以为您推荐适合的款式。”



In [11]:
db.save_local("real_estates_sale")

### 使用 retriever 从向量数据库中获取结果

#### 使用参数 `k` 指定返回结果数量


In [12]:
# 实例化一个 TopK Retriever
topK_retriever = db.as_retriever(search_kwargs={"k": 3})

In [13]:
topK_retriever

VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000002793D4BC210>, search_kwargs={'k': 3})

In [14]:
docs = topK_retriever.get_relevant_documents(query)
for doc in docs:
    print(doc.page_content + "\n")

[客户问题] “我担心这件外套的质量不够好。”
   [销售回答] “我们的外套采用了优质的材料，经过严格的质量控制。此外，我们还提供了一年的质量保证，如果您在使用过程中有任何问题，我们都会为您提供全额退款或换货。”

[客户问题] “我对这个款式的外套的保暖性有些疑问，你能介绍一下吗？”
    [销售回答] “这款外套采用了保暖性能较好的面料，内部可能还有保暖填充物，确保在寒冷的天气中保持温暖。您也可以选择适合的内衬来增加保暖效果，让您在冬季穿着更加舒适。”

[客户问题] “我对这个款式的衣服的质地有些担心，你能介绍一下吗？”
    [销售回答] “这款衣服采用了优质的面料，手感柔软舒适，具有良好的透气性和吸湿排汗性能，穿着舒适且质地优良。我们也提供了清洗说明，让您更好地保养衣服。”



In [15]:
docs = topK_retriever.get_relevant_documents("有没有适合约会的连衣裙")

In [16]:
for doc in docs:
    print(doc.page_content + "\n")

[客户问题] “我想要购买一些适合约会穿着的服装，你有什么推荐吗？”
    [销售回答] “约会穿着需要既有个性又具有魅力，我们有多款适合约会穿着的服装，如性感连衣裙、时尚牛仔裤、迷人裙装等，让您在约会中散发迷人魅力。”

[客户问题] “我是个经常参加社交活动的人，需要一些适合社交场合穿着的服装。”
    [销售回答] “社交场合需要着装得体且引人注目，我们有多款适合社交场合穿着的服装，无论是晚礼服、派对装还是优雅连衣裙，都能让您在社交活动中脱颖而出。”

[客户问题] “我希望购买一些适合参加婚礼穿着的礼服，有什么推荐吗？”
    [销售回答] “参加婚礼需要庄重得体的礼服，我们有多款适合参加婚礼穿着的礼服，如优雅晚礼服、精致长裙、高贵连衣裙等，让您在婚礼上成为焦点。”



#### 使用 similarity_score_threshold 设置阈值，提升结果的相关性质量

In [17]:
# 实例化一个 similarity_score_threshold Retriever
retriever = db.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={"score_threshold": 0.8}
)

In [18]:
docs = retriever.get_relevant_documents(query)
for doc in docs:
    print(doc.page_content + "\n")

[客户问题] “我担心这件外套的质量不够好。”
   [销售回答] “我们的外套采用了优质的材料，经过严格的质量控制。此外，我们还提供了一年的质量保证，如果您在使用过程中有任何问题，我们都会为您提供全额退款或换货。”

[客户问题] “我对这个款式的外套的保暖性有些疑问，你能介绍一下吗？”
    [销售回答] “这款外套采用了保暖性能较好的面料，内部可能还有保暖填充物，确保在寒冷的天气中保持温暖。您也可以选择适合的内衬来增加保暖效果，让您在冬季穿着更加舒适。”



### 提取向量数据库中的`销售回答`

In [19]:
docs = retriever.get_relevant_documents(query)

In [20]:
docs[0].page_content

'[客户问题] “我担心这件外套的质量不够好。”\n   [销售回答] “我们的外套采用了优质的材料，经过严格的质量控制。此外，我们还提供了一年的质量保证，如果您在使用过程中有任何问题，我们都会为您提供全额退款或换货。”'

In [21]:
docs[0].page_content.split("[销售回答] ")

['[客户问题] “我担心这件外套的质量不够好。”\n   ',
 '“我们的外套采用了优质的材料，经过严格的质量控制。此外，我们还提供了一年的质量保证，如果您在使用过程中有任何问题，我们都会为您提供全额退款或换货。”']

In [22]:
ans = docs[0].page_content.split("[销售回答] ")[-1]

In [23]:
ans

'“我们的外套采用了优质的材料，经过严格的质量控制。此外，我们还提供了一年的质量保证，如果您在使用过程中有任何问题，我们都会为您提供全额退款或换货。”'

#### 尝试各种问题

In [24]:
from typing import List

def sales(query: str, score_threshold: float=0.8) -> List[str]:
    retriever = db.as_retriever(search_type="similarity_score_threshold", search_kwargs={"score_threshold": score_threshold})    
    docs = retriever.get_relevant_documents(query)
    ans_list = [doc.page_content.split("[销售回答] ")[-1] for doc in docs]

    return ans_list

In [25]:
query = "我想要运动套装"

print(sales(query))

['“运动健身需要舒适透气且具有弹性的服装，我们有多款适合运动健身穿着的服装，如运动上衣、运动裤、运动内衣等，让您在运动中保持舒适和自由。”', '“运动健身需要舒适透气的服装，同时具备良好的伸缩性和吸汗排湿功能。我们有各种款式的运动服装，包括运动T恤、运动裤、运动文胸等，保证您在运动中保持舒适和自信。”', '“我们有多款适合运动休闲穿着的服装，如运动T恤、休闲裤、运动套装等，舒适透气，让您在运动和休闲时都能保持时尚和舒适。”', '“夏季运动服需要轻薄透气，同时具备良好的排汗性能。我们有多种款式和材质的运动服可供选择，您可以根据您的喜好和运动类型来挑选。此外，我们的运动服还有防紫外线和快干功能，确保您在运动中保持舒适。”']


In [26]:
print(sales(query, 0.75))

['“运动健身需要舒适透气且具有弹性的服装，我们有多款适合运动健身穿着的服装，如运动上衣、运动裤、运动内衣等，让您在运动中保持舒适和自由。”', '“运动健身需要舒适透气的服装，同时具备良好的伸缩性和吸汗排湿功能。我们有各种款式的运动服装，包括运动T恤、运动裤、运动文胸等，保证您在运动中保持舒适和自信。”', '“我们有多款适合运动休闲穿着的服装，如运动T恤、休闲裤、运动套装等，舒适透气，让您在运动和休闲时都能保持时尚和舒适。”', '“夏季运动服需要轻薄透气，同时具备良好的排汗性能。我们有多种款式和材质的运动服可供选择，您可以根据您的喜好和运动类型来挑选。此外，我们的运动服还有防紫外线和快干功能，确保您在运动中保持舒适。”']


In [27]:
query = "价格200万以内"

print(f"score:0.8 ans: {sales(query)}\n")
print(f"score:0.75 ans: {sales(query, 0.75)}\n")
print(f"score:0.5 ans: {sales(query, 0.5)}\n")

C:\Users\ckc\.conda\envs\langchain\Lib\site-packages\langchain_core\vectorstores.py:342: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.8
  warnings.warn(


score:0.8 ans: []



C:\Users\ckc\.conda\envs\langchain\Lib\site-packages\langchain_core\vectorstores.py:342: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.75
  warnings.warn(


score:0.75 ans: []

score:0.5 ans: ['“我们真诚地感谢您对我们产品的关注，并理解您对价格的考量。尽管我们竭尽所能确保提供最具竞争力的价格，但我们也重视产品质量和服务。我们会尽最大努力为您提供更好的购物体验，包括提供专业的服务和优质的售后支持。”', '“我们的产品价格根据款式和材质的不同而有所差异，价格范围也比较灵活。我们有适合不同消费水平的产品线，您可以根据自己的需求和预算选择适合的产品。”', '“我明白您关注价格的考量。但是考虑到我们的产品质量和设计，这个价格实际上是非常合理的。不过，如果您对其他款式感兴趣，我可以帮您找到一些更优惠的选择。”', '“我们的外套采用了优质的材料，经过严格的质量控制。此外，我们还提供了一年的质量保证，如果您在使用过程中有任何问题，我们都会为您提供全额退款或换货。”']



#### 当向量数据库中没有合适答案时，使用大语言模型能力

In [28]:
from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.5)
qa_chain = RetrievalQA.from_chain_type(llm,
                                       retriever=db.as_retriever(search_type="similarity_score_threshold",
                                                                 search_kwargs={"score_threshold": 0.8}))

In [29]:
qa_chain({"query": "你们的衣服舒适吗"})

C:\Users\ckc\.conda\envs\langchain\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


{'query': '你们的衣服舒适吗',
 'result': '是的，我们有多款舒适度较高的服装，如棉质T恤、羊毛衫、弹力裤等，面料柔软舒适，剪裁合身，让您在穿着时感到舒适自在。'}

In [30]:
qa_chain({"query": "你们的西装怎么样"})

C:\Users\ckc\.conda\envs\langchain\Lib\site-packages\langchain_core\vectorstores.py:342: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.8
  warnings.warn(


{'query': '你们的西装怎么样', 'result': '很抱歉，我不清楚你在问哪家公司的西装。你可以提供更多细节吗？'}

In [31]:
print(sales("你们的退货服务怎么样"))

['“我们有非常灵活的退换货政策，您可以放心购买。如果您对商品不满意或尺码不合适，我们接受无条件退货并提供全额退款。如果您更换其他商品，我们也会尽力满足您的需求。”']


## 加载 FAISS 向量数据库已有结果

In [32]:
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS

db = FAISS.load_local("real_estates_sale", OpenAIEmbeddings(),allow_dangerous_deserialization = True)

In [33]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.5)
qa_chain = RetrievalQA.from_chain_type(llm,
                                       retriever=db.as_retriever(search_type="similarity_score_threshold",
                                                                 search_kwargs={"score_threshold": 0.8}))

C:\Users\ckc\.conda\envs\langchain\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [34]:
qa_chain({"query": "你们的运动装舒适吗"})

{'query': '你们的运动装舒适吗',
 'result': '根据提供的信息，我们的运动装设计舒适透气，具有良好的伸缩性和吸汗排湿功能，让您在运动中保持舒适和自由。'}

In [35]:
# 输出内部 Chain 的日志
qa_chain.combine_documents_chain.verbose = True

In [36]:
qa_chain({"query": "你们的运动装舒适吗"})



> Entering new StuffDocumentsChain chain...

> Finished chain.


{'query': '你们的运动装舒适吗',
 'result': '根据提供的上下文，销售员强调他们的运动装舒适透气、具有弹性和吸汗排湿功能，因此可以说他们的运动装应该是舒适的。'}

In [37]:
# 返回向量数据库的检索结果
qa_chain.return_source_documents = True

In [38]:
result = qa_chain({"query": "你们的运动装舒适吗"})



> Entering new StuffDocumentsChain chain...

> Finished chain.


In [39]:
result

{'query': '你们的运动装舒适吗',
 'result': '根据提供的信息，我们的运动装设计为舒适透气、具有弹性和吸汗排湿功能，以确保您在运动中感到舒适和自由。',
 'source_documents': [Document(page_content='[客户问题] “我希望购买一些适合运动健身穿着的服装，有什么推荐吗？”\n    [销售回答] “运动健身需要舒适透气且具有弹性的服装，我们有多款适合运动健身穿着的服装，如运动上衣、运动裤、运动内衣等，让您在运动中保持舒适和自由。”'),
  Document(page_content='[客户问题] “我想购买一些适合运动健身穿着的服装。”\n    [销售回答] “运动健身需要舒适透气的服装，同时具备良好的伸缩性和吸汗排湿功能。我们有各种款式的运动服装，包括运动T恤、运动裤、运动文胸等，保证您在运动中保持舒适和自信。”')]}